In [26]:
from compute_signals import *
from process_signal_data import *
from model_classifier import *
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import random
from scipy.interpolate import CubicSpline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
from scipy.signal import welch, butter, filtfilt, hilbert
import numpy as np
import os


- lf is the frequency band for the Low frequency from the paper
- hf is the frequency band for the High frequency from the paper
- percent_outliers is the fraction of values to remove from the lower and upper values of the window

- We will load the the dataframe that is provided and then subsequently add the feature columns to it

In [2]:
data = pd.read_csv('userData.csv')
# data.head()

In [3]:
lf =  [0.04, 0.15]
hf = [0.15, 0.4]
percent_outliers = 0.1
plot_flag = 1
save_fig = 1
kernel_type = 'poly'
window_size =  90
offset_window = 10
sample_points = 5


- **convert_rr_total** will convert the **rr** column to arrays and save it as a column in the dataframe
- **compute_sampling_rate** will compute the sampling rate for each sample and then save it as a column

In [4]:
convert_rr_total(data)
compute_sampling_rate(data)
data.head(15)

,user,age,gender,section,duration,arousal,valence,samples,rr,rr_array,fs
0,20230608_183012_3_02001743,42,0,Rest_Start,185,3,3,215,"[783, 790, 795, 769, 768, 754, 762, 766, 774, ...","[783.0, 790.0, 795.0, 769.0, 768.0, 754.0, 762...",1.162162
1,20230608_183012_3_02001743,42,0,Reading,185,4,2,207,"[739, 749, 724, 692, 701, 724, 757, 769, 777, ...","[739.0, 749.0, 724.0, 692.0, 701.0, 724.0, 757...",1.118919
2,20230608_183012_3_02001743,42,0,Breath_1,130,5,3,155,"[750, 751, 735, 741, 749, 760, 752, 761, 774, ...","[750.0, 751.0, 735.0, 741.0, 749.0, 760.0, 752...",1.192308
3,20230608_183012_3_02001743,42,0,Stroop,190,3,0,220,"[753, 765, 778, 757, 768, 774, 775, 773, 777, ...","[753.0, 765.0, 778.0, 757.0, 768.0, 774.0, 775...",1.157895
4,20230608_183012_3_02001743,42,0,ReactionTime,227,5,3,241,"[698, 741, 791, 814, 813, 827, 849, 870, 820, ...","[698.0, 741.0, 791.0, 814.0, 813.0, 827.0, 849...",1.061674
5,20230608_183012_3_02001743,42,0,Video+,179,5,3,177,"[780, 786, 785, 775, 770, 762, 749, 762, 769, ...","[780.0, 786.0, 785.0, 775.0, 770.0, 762.0, 749...",0.988827
6,20230608_183012_3_02001743,42,0,Breath_2,131,6,4,131,"[778, 773, 767, 761, 741, 749, 747, 758, 773, ...","[778.0, 773.0, 767.0, 761.0, 741.0, 749.0, 747...",1.000000
7,20230608_183012_3_02001743,42,0,Math,190,3,2,187,"[768, 761, 738, 723, 723, 712, 727, 724, 731, ...","[768.0, 761.0, 738.0, 723.0, 723.0, 712.0, 727...",0.984211
8,20230608_183012_3_02001743,42,0,ImageStroop,294,6,4,297,"[706, 706, 700, 716, 717, 743, 741, 737, 748, ...","[706.0, 706.0, 700.0, 716.0, 717.0, 743.0, 741...",1.010204
9,20230608_183012_3_02001743,42,0,Video-,162,3,1,187,"[720, 715, 732, 723, 708, 717, 719, 728, 703, ...","[720.0, 715.0, 732.0, 723.0, 708.0, 717.0, 719...",1.154321


In [6]:
dict_new =  {'user':users_unique,'section':None}
df_breath1 = pd.DataFrame.from_dict(dict_new)
df_breath2 = pd.DataFrame.from_dict(dict_new)
df_breath3 = pd.DataFrame.from_dict(dict_new)
df_breath4 = pd.DataFrame.from_dict(dict_new)

df_mental = pd.DataFrame.from_dict(dict_new)
df_rest_start = pd.DataFrame.from_dict(dict_new)
df_rest_end = pd.DataFrame.from_dict(dict_new)
df_stroop = pd.DataFrame.from_dict(dict_new)
df_imagestroop = pd.DataFrame.from_dict(dict_new)
df_math = pd.DataFrame.from_dict(dict_new)
df_video_plus = pd.DataFrame.from_dict(dict_new)
df_video_minus = pd.DataFrame.from_dict(dict_new)
df_breath_total = pd.DataFrame.from_dict(dict_new)
df_rest_total  = pd.DataFrame.from_dict(dict_new)
df_reaction_time  = pd.DataFrame.from_dict(dict_new)
df_reading  = pd.DataFrame.from_dict(dict_new)

df_exercise = pd.DataFrame.from_dict(dict_new)
# a step to make empty dataframes for each user and each section


In [8]:
stroop_vals = construct_signal_general(data,users_unique,['Stroop'])
math_vals = construct_signal_general(data,users_unique,['Math'])
video_minus_vals = construct_signal_general(data,users_unique,['Video-'])
video_plus_vals = construct_signal_general(data,users_unique,['Video+'])
imagestroop_vals = construct_signal_general(data,users_unique,['ImageStroop'])
breath1_vals = construct_signal_general(data,users_unique,['Breath_1'])
breath2_vals = construct_signal_general(data,users_unique,['Breath_2'])
breath3_vals = construct_signal_general(data,users_unique,['Breath_3'])
breath4_vals = construct_signal_general(data,users_unique,['Breath_4'])
breath_total_val = construct_signal_general(data,users_unique,['Breath_1','Breath_2','Breath_3', 'Breath_4'])
rest_start_vals = construct_signal_general(data,users_unique,['Rest_Start'])
rest_end_vals = construct_signal_general(data,users_unique,['Rest_End'])
rest_total_vals = construct_signal_general(data,users_unique,['Rest_Start','Rest_End'])
reactiontime_vals = construct_signal_general(data,users_unique,['ReactionTime'])
reading_vals = construct_signal_general(data,users_unique,['Reading'])


df_breath1['signal'] = breath1_vals
df_breath2['signal'] = breath2_vals
df_breath3['signal'] = breath3_vals
df_breath4['signal'] = breath4_vals
df_breath_total['signal'] = breath_total_val

df_rest_start['signal'] = rest_start_vals
df_rest_end['signal'] = rest_end_vals
df_rest_total['signal'] = rest_total_vals
df_stroop['signal'] = stroop_vals
df_imagestroop['signal'] = imagestroop_vals
df_math['signal'] = math_vals
df_video_plus['signal'] = video_plus_vals
df_video_minus['signal'] = video_minus_vals
df_reaction_time['signal'] = reactiontime_vals
df_reading['signal'] = reading_vals


# i feel like this is being done to make separate dataframes for each activity which is lost to me at the moment - so lets see 

In [10]:
stroop_fs,stroop_samples, stroop_duration = construct_fs_general(data,users_unique,['Stroop'])
math_fs,math_samples, math_duration = construct_fs_general(data,users_unique,['Math'])
videoplus_fs,videoplus_samples, videoplus_duration = construct_fs_general(data,users_unique,['Video+'])
videominus_fs,videominus_samples, videominus_duration = construct_fs_general(data,users_unique,['Video-'])
imagestroop_fs,imagestroop_samples, imagestroop_duration = construct_fs_general(data,users_unique,['ImageStroop'])
breath1_fs,breath1_samples, breath1_duration = construct_fs_general(data,users_unique,['Breath_1'])
breath2_fs,breath2_samples, breath2_duration = construct_fs_general(data,users_unique,['Breath_2'])
breath3_fs,breath3_samples, breath3_duration = construct_fs_general(data,users_unique,['Breath_3'])
breath4_fs,breath4_samples, breath4_duration = construct_fs_general(data,users_unique,['Breath_4'])
breathtotal_fs,breathtotal_samples, breathtotal_duration = construct_fs_general(data,users_unique, ['Breath_1','Breath_2','Breath_3', 'Breath_4'])

rest_start_fs,rest_start_samples, rest_start_duration = construct_fs_general(data,users_unique,['Rest_Start'])
rest_end_fs,rest_end_samples, rest_end_duration= construct_fs_general(data,users_unique,['Rest_End'])
rest_total_fs,rest_total_samples, rest_total_duration= construct_fs_general(data, users_unique,['Rest_Start','Rest_End'])
reaction_fs,reaction_samples, reaction_duration= construct_fs_general(data, users_unique,['ReactionTime'])
reading_fs,reading_samples, reading_duration= construct_fs_general(data, users_unique,['Reading'])


In [13]:
df_breath1['fs'] = np.array(breath1_fs)
df_breath2['fs'] = np.array(breath2_fs)
df_breath3['fs'] = np.array(breath3_fs)
df_breath4['fs'] = np.array(breath4_fs)
df_breath_total['fs'] = np.array(breathtotal_fs)


df_rest_start['fs'] = np.array(rest_start_fs)
df_rest_end['fs'] = np.array(rest_end_fs)
df_rest_total['fs'] = np.array(rest_total_fs)

df_stroop['fs'] = np.array(stroop_fs)
df_imagestroop['fs'] = np.array(imagestroop_fs)
df_math['fs'] = np.array(math_fs)
df_video_plus['fs'] = np.array(videoplus_fs)
df_video_minus['fs'] = np.array(videominus_fs)
df_reaction_time['fs'] = np.array(reaction_fs)
df_reading['fs'] = np.array(reading_fs)


In [15]:
df_breath1['samples'] = np.array(breath1_samples)
df_breath2['samples'] = np.array(breath2_samples)
df_breath3['samples'] = np.array(breath3_samples)
df_breath4['samples'] = np.array(breath4_samples)
df_breath_total['samples'] = np.array(breathtotal_samples)


df_rest_start['samples'] = np.array(rest_start_samples)
df_rest_end['samples'] = np.array(rest_end_samples)
df_rest_total['samples'] = np.array(rest_total_samples)

df_stroop['samples'] = np.array(stroop_samples)
df_imagestroop['samples'] = np.array(imagestroop_samples)
df_math['samples'] = np.array(math_samples)
df_video_plus['samples'] = np.array(videoplus_samples)
df_video_minus['samples'] = np.array(videominus_samples)
df_reaction_time['samples'] = np.array(reaction_samples)
df_reading['samples'] = np.array(reading_samples)

In [17]:
df_breath1['duration'] = np.array(breath1_duration)
df_breath2['duration'] = np.array(breath2_duration)
df_breath3['duration'] = np.array(breath3_duration)
df_breath4['duration'] = np.array(breath4_duration)
df_breath_total['duration'] = np.array(breathtotal_duration)

df_rest_start['duration'] = np.array(rest_start_duration)
df_rest_end['duration'] = np.array(rest_end_duration)
df_rest_total['duration'] = np.array(rest_total_duration)

df_stroop['duration'] = np.array(stroop_duration)
df_imagestroop['duration'] = np.array(imagestroop_duration)
df_math['duration'] = np.array(math_duration)
df_video_plus['duration'] = np.array(videoplus_duration)
df_video_minus['duration'] = np.array(videominus_duration)
df_reaction_time['duration'] = np.array(reaction_duration)
df_reading['duration'] = np.array(reading_duration)

In [19]:
# combing all the dataframes into one big dataframe from this point onwards i will be writing my own code and not using the one provided originally
# List of your dataframes and their corresponding activities
dataframes = {
    "breath_total": df_breath_total,
    "rest_total": df_rest_total,
    "stroop": df_stroop,
    "imagestroop": df_imagestroop,
    "math": df_math,
    "video_plus": df_video_plus,
    "video_minus": df_video_minus,
    "reaction_time": df_reaction_time,
    "reading": df_reading
}

# Adding the 'activity' column to each dataframe
for activity, df in dataframes.items():
    df['activity'] = activity

# Concatenating all dataframes
main_df = pd.concat(dataframes.values(), ignore_index=True)
main_df.head()

#drop column section in the main dataframe
main_df = main_df.drop(columns=['section'])
main_df.head()

,user,signal,fs,samples,duration,activity
0,20230608_183012_3_02001743,"[750.0, 751.0, 735.0, 741.0, 749.0, 760.0, 752...",0.928983,484,521,breath_total
1,20230608_210755_4_02001743,"[712.0, 716.0, 729.0, 681.0, 668.0, 672.0, 628...",1.126923,586,520,breath_total
2,20230609_180159_5_02001743,"[631.0, 638.0, 655.0, 731.0, 681.0, 676.0, 715...",1.177820,616,523,breath_total
3,20230609_230133_6_02001743,"[719.0, 719.0, 720.0, 731.0, 701.0, 711.0, 706...",1.024952,534,521,breath_total
4,20230613_173426_9_02001743,"[629.0, 631.0, 627.0, 634.0, 626.0, 630.0, 625...",1.262452,659,522,breath_total


In [20]:
type(main_df['signal'][0])

numpy.ndarray

In [21]:
# Computing hf and lf ussing fft and not the butter_band pass filter used in the paper redefinig the window as mentioned in the paper 
# Function to compute LF, HF, and LF/HF ratio
def compute_hrv_metrics(rr_intervals, fs):
    # Calculate power spectral density using Welch's method
    f, pxx = welch(rr_intervals, fs=fs, nperseg=len(rr_intervals))

    # Frequency bands for LF and HF components
    lf_band = (0.04, 0.15)
    hf_band = (0.15, 0.4)

    # Calculate power in LF and HF bands
    lf_power = np.trapz(pxx[(f >= lf_band[0]) & (f <= lf_band[1])], f[(f >= lf_band[0]) & (f <= lf_band[1])])
    hf_power = np.trapz(pxx[(f >= hf_band[0]) & (f <= hf_band[1])], f[(f >= hf_band[0]) & (f <= hf_band[1])])

    # Calculate LF/HF ratio
    lf_hf_ratio = lf_power / hf_power

    return lf_power, hf_power, lf_hf_ratio

# Sliding window function
def sliding_window_analysis(rr_intervals, fs, window_length_sec=120, increment_sec=10): # window length is 2 minutes and increment is 10 seconds
    window_samples = int(window_length_sec * fs)
    increment_samples = int(increment_sec * fs)
    n_samples = len(rr_intervals)

    if n_samples < window_samples:  # If the signal is shorter than the window size
        return compute_hrv_metrics(rr_intervals, fs)  # Compute metrics for the entire signal

    # Calculate the number of windows
    n_windows = int(np.floor((n_samples - window_samples) / increment_samples) + 1)

    # Lists to store results for each window
    lf_values = []
    hf_values = []
    lf_hf_ratio_values = []

    # Slide the window and compute metrics
    for i in range(n_windows):
        start_idx = i * increment_samples
        end_idx = start_idx + window_samples
        windowed_rr = rr_intervals[start_idx:end_idx]

        lf, hf, lf_hf = compute_hrv_metrics(windowed_rr, fs)
        lf_values.append(lf)
        hf_values.append(hf)
        lf_hf_ratio_values.append(lf_hf)

    return lf_values, hf_values, lf_hf_ratio_values

In [22]:
# Function to process each row in the dataframe
def process_row(row):
    rr_intervals = row['signal']
    fs = row['fs']
    lf, hf, lf_hf = sliding_window_analysis(rr_intervals, fs)
    return pd.Series([lf, hf, lf_hf])


In [23]:
# updating the main_df
main_df[['LFp', 'HFp', 'LF_HF']] = main_df.apply(process_row, axis=1)


In [46]:
 # Example values for fs, lowcut, and highcut
fs_example = 1.3  #sanity check
lowcut = 0.04
highcut = 0.4

# iterate through all fs values in the main_df and mulitply with nyq then divde lowcut and highcut by nyq and save them in a list low and high respectively 
low = []
high = []
for i in range(len(main_df['fs'])):
    nyq = 0.5 * main_df['fs'][i]
    low.append(lowcut / nyq)
    high.append(highcut / nyq)

#check if the values are properly normalized and all values in both list are between 0 and 1 if not then print and save them in lists low_ind and high_ind the index of the value that is not normalized
low_ind = []
high_ind = []
for i in range(len(low)):
    if low[i] < 0 or low[i] > 1:
        low_ind.append(i)
    if high[i] < 0 or high[i] > 1:
        high_ind.append(i)
print(low_ind)
print(high_ind)

# #iterate through the high_ind to print the values of fs and highcut that are not normalized
# for i in range(len(high_ind)):
#     print(main_df['fs'][high_ind[i]])
#     print(highcut)



[]
[16, 19, 26, 45, 56, 59, 66, 96, 106, 108, 114, 134, 136, 139, 146, 176, 179, 186, 205, 214, 216, 219, 226, 245, 256, 259, 272, 296, 314, 334, 336, 346]


In [51]:
# Filter out rows where the highcut frequency, when normalized, would be >= 1
min_fs_required = 2 * highcut  # Twice the highcut to satisfy Nyquist
main_df_filtered = main_df[main_df['fs'] > min_fs_required]
#reset index
main_df_filtered = main_df_filtered.reset_index(drop=True)

In [52]:
 # Example values for fs, lowcut, and highcut
fs_example = 1.3  #sanity check
lowcut = 0.04
highcut = 0.4

# iterate through all fs values in the main_df and mulitply with nyq then divde lowcut and highcut by nyq and save them in a list low and high respectively 
low = []
high = []
for i in range(len(main_df_filtered['fs'])):
    nyq = 0.5 * main_df_filtered['fs'][i]
    low.append(lowcut / nyq)
    high.append(highcut / nyq)

#check if the values are properly normalized and all values in both list are between 0 and 1 if not then print and save them in lists low_ind and high_ind the index of the value that is not normalized
low_ind = []
high_ind = []
for i in range(len(low)):
    if low[i] < 0 or low[i] > 1:
        low_ind.append(i)
    if high[i] < 0 or high[i] > 1:
        high_ind.append(i)
print(low_ind)
print(high_ind)

# #iterate through the high_ind to print the values of fs and highcut that are not normalized
# for i in range(len(high_ind)):
#     print(main_df['fs'][high_ind[i]])
#     print(highcut)



[]
[]


In [53]:
# Making new dataframe with just the user and activity columns and lf_hf columns will use this later for the svm approach and comparison 
df_P1_lf_hf = main_df_filtered[['user', 'activity', 'LF_HF']]
df_P1_lf_hf.head()
# print(df_P1_lf_hf.shape)

,user,activity,LF_HF
0,20230608_183012_3_02001743,breath_total,"[4.578940708636031, 5.039292189476101, 6.30402..."
1,20230608_210755_4_02001743,breath_total,"[1.247832100030658, 0.8930347441208322, 0.9557..."
2,20230609_180159_5_02001743,breath_total,"[6.024086186862033, 6.429796170979665, 6.06763..."
3,20230609_230133_6_02001743,breath_total,"[0.5551263783555743, 0.5742567524978311, 0.570..."
4,20230613_173426_9_02001743,breath_total,"[0.44626080967528686, 0.4456185823002845, 0.47..."


In [54]:
# the butterbandpass filter is the same as the original one provided the only reason it is being called out here is since we changed the window size and offset window size
# and added the outlier removal function in one go this is the 2d part of the paper

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs  # Nyquist frequency
    low = lowcut / nyq  # Normalize the low cutoff frequency
    high = highcut / nyq  # Normalize the high cutoff frequency
    if low <= 0 or high >= 1 or low >= high:
        raise ValueError("The cutoff frequencies must satisfy 0 < low < high < Nyquist")
    b, a = butter(order, [low, high], btype='band')  # Use normalized frequencies
    return b, a
    
# Function to bandpass-filter the signal
def bandpass_filter_signal(signal, fs, lowcut, highcut, order=2):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)  # Use the corrected butter_bandpass function
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

# Function to apply Hilbert transform and get instantaneous amplitude
def hilbert_transform(signal):
    analytic_signal = hilbert(signal)  # Apply Hilbert transform
    amplitude_envelope = np.abs(analytic_signal)  # Get instantaneous amplitude
    return amplitude_envelope

# Sliding window analysis with outlier removal and mean calculation
def sliding_window_analysis(signal, fs, window_length_sec=120, increment_sec=10):
    # Calculate the number of samples in the window and increment
    window_samples = int(window_length_sec * fs)
    increment_samples = int(increment_sec * fs)
    
    # Calculate the number of windows
    n_samples = len(signal)
    n_windows = int(np.floor((n_samples - window_samples) / increment_samples) + 1)
    
    # Initialize list to store mean values for each window
    windowed_means = []

    # Slide the window and compute metrics
    for i in range(n_windows):
        start_idx = i * increment_samples
        end_idx = start_idx + window_samples
        windowed_signal = signal[start_idx:end_idx]

        # Remove the 20% largest and smallest values
        windowed_signal_sorted = np.sort(windowed_signal)
        n_20_percent = int(0.2 * len(windowed_signal_sorted))
        windowed_signal_trimmed = windowed_signal_sorted[n_20_percent:-n_20_percent]
        
        # Calculate the mean of the trimmed signal
        mean_value = np.mean(windowed_signal_trimmed)
        windowed_means.append(mean_value)

    return windowed_means

# Apply to each row of the dataframe
def process_row(row):
    rr_intervals = row['signal']
    fs = row['fs']
    # Filter the signal into LF and HF bands
    lf_signal = bandpass_filter_signal(rr_intervals, fs, 0.04, 0.15)
    hf_signal = bandpass_filter_signal(rr_intervals, fs, 0.15, 0.4)
    # Apply Hilbert transform and get iA
    lf_ia = hilbert_transform(lf_signal)
    hf_ia = hilbert_transform(hf_signal)
    # Perform sliding window analysis
    lf_ia_mean = sliding_window_analysis(lf_ia, fs)
    hf_ia_mean = sliding_window_analysis(hf_ia, fs)
    return pd.Series([lf_ia_mean, hf_ia_mean])

In [56]:
# Apply the function to each row and store the results in new columns
main_df_filtered[['LFia', 'HFia']] = main_df_filtered.apply(process_row, axis=1)
main_df_filtered.head()

,user,signal,fs,samples,duration,activity,LFp,HFp,LF_HF,LFia,HFia
0,20230608_183012_3_02001743,"[750.0, 751.0, 735.0, 741.0, 749.0, 760.0, 752...",0.928983,484,521,breath_total,"[2301.0869878925896, 2231.8876935763155, 2415....","[502.53696964292743, 442.8970596778094, 383.14...","[4.578940708636031, 5.039292189476101, 6.30402...","[62.63175333681161, 67.14820173335953, 67.8861...","[13.105872439196567, 15.057671311078881, 14.98..."
1,20230608_210755_4_02001743,"[712.0, 716.0, 729.0, 681.0, 668.0, 672.0, 628...",1.126923,586,520,breath_total,"[8307.910168911769, 5833.717056588233, 5226.80...","[6657.875020772147, 6532.463708711989, 5468.84...","[1.247832100030658, 0.8930347441208322, 0.9557...","[117.24347583905454, 120.07741411009444, 125.0...","[49.30159777302439, 45.071411563526254, 47.155..."
2,20230609_180159_5_02001743,"[631.0, 638.0, 655.0, 731.0, 681.0, 676.0, 715...",1.177820,616,523,breath_total,"[7563.10692243634, 8017.792291627798, 8418.010...","[1255.4778746244976, 1246.974566288029, 1387.3...","[6.024086186862033, 6.429796170979665, 6.06763...","[111.54849471512217, 112.44251168329953, 112.4...","[31.36543351350337, 31.66538885079125, 33.9216..."
3,20230609_230133_6_02001743,"[719.0, 719.0, 720.0, 731.0, 701.0, 711.0, 706...",1.024952,534,521,breath_total,"[7934.139460619775, 9445.968902787097, 9406.29...","[14292.492250364172, 16449.03409790166, 16493....","[0.5551263783555743, 0.5742567524978311, 0.570...","[50.124183344365115, 56.5920711660148, 73.4288...","[34.533560426437795, 40.33041688226922, 57.631..."
4,20230613_173426_9_02001743,"[629.0, 631.0, 627.0, 634.0, 626.0, 630.0, 625...",1.262452,659,522,breath_total,"[916.6418810097641, 523.2093547478282, 248.539...","[2054.049697253812, 1174.1192480058169, 525.73...","[0.44626080967528686, 0.4456185823002845, 0.47...","[20.79935035613921, 21.107786818647025, 21.080...","[13.46250778948595, 13.073058961065392, 12.520..."
